# Check for feasibility

In [1]:
import glob
import os
import tensorflow as tf
import cv2
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.models import Sequential

## Set hyperparameter

In [13]:
EPOCHS = 50

DATASET_PATH = '/Users/shim/dl-python-ImageDetection/dataset/2/'
DATASET_OK_PATTERN = '/Users/shim/dl-python-ImageDetection/dataset/2/OK/'
DATASET_FAIL_PATTERN = '/Users/shim/dl-python-ImageDetection/dataset/2/FAIL/'

RESULT_SAVE_PATH = '/Users/shim/dl-python-ImageDetection/dataset/results/'

## Simple Model Setting 

In [3]:
def Model():
    return Sequential([Conv2D(32, (3,3), activation='relu',input_shape=(256, 256, 1)),
                       MaxPool2D(),
                       Conv2D(64, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(128, (3,3), activation='relu'),
                       MaxPool2D(),
                       Conv2D(256, (3,3), activation='relu'),
                       MaxPool2D(),
                       Flatten(),
                       Dense(1, activation='sigmoid')])

## Import dataset

In [5]:
def preprocess(file_name):
    img = tf.io.read_file(file_name)
    img = tf.image.decode_png(img, channels=1)
    return tf.image.convert_image_dtype(img, tf.float32)

In [23]:
def get_file_list(directory, extension):
    file_list = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_list.append(os.path.join(root, file))
    return file_list


In [14]:
ok_list = get_file_list(DATASET_OK_PATTERN, '.png')
ds_ok = tf.data.Dataset.list_files(ok_list)
ds_ok_label = tf.data.Dataset.from_tensor_slices([0] * len(ok_list))

ds_ok = ds_ok.map(preprocess)
ds_ok = tf.data.Dataset.zip((ds_ok, ds_ok_label))

fail_list = get_file_list(DATASET_FAIL_PATTERN, '.png')
ds_fail = tf.data.Dataset.list_files(fail_list)
ds_fail_label = tf.data.Dataset.from_tensor_slices([1] * len(fail_list))

ds_fail = ds_fail.map(preprocess)
ds_fail = tf.data.Dataset.zip((ds_fail, ds_fail_label))

ds = tf.data.Dataset.concatenate(ds_ok, ds_fail)

## Split Train, Validation set

In [15]:
ds_size = len(ok_list) + len(fail_list)
train_size = int(ds_size * 0.7)

ds = ds.shuffle(ds_size)
ds_train = ds.take(train_size).shuffle(1024, reshuffle_each_iteration=True).batch(32)
ds_validation = ds.skip(train_size).batch(32)

In [17]:
train_size

662

## Build and Train the Model

In [18]:
model = Model()
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [19]:
model.fit(ds_train, validation_data = ds_validation, epochs=EPOCHS)

Epoch 1/50
21/21 [==============================] - 32s 1s/step - loss: 0.4665 - accuracy: 0.8202 - val_loss: 0.3626 - val_accuracy: 0.8807
Epoch 2/50
21/21 [==============================] - 33s 2s/step - loss: 0.3879 - accuracy: 0.8731 - val_loss: 0.3644 - val_accuracy: 0.8632
Epoch 3/50
21/21 [==============================] - 35s 2s/step - loss: 0.3227 - accuracy: 0.8882 - val_loss: 0.3242 - val_accuracy: 0.8842
Epoch 4/50
21/21 [==============================] - 37s 2s/step - loss: 0.3510 - accuracy: 0.8776 - val_loss: 0.3232 - val_accuracy: 0.8947
Epoch 5/50
21/21 [==============================] - 38s 2s/step - loss: 0.3166 - accuracy: 0.8943 - val_loss: 0.3579 - val_accuracy: 0.8807
Epoch 6/50
21/21 [==============================] - 38s 2s/step - loss: 0.3527 - accuracy: 0.8807 - val_loss: 0.3753 - val_accuracy: 0.8316
Epoch 7/50
21/21 [==============================] - 39s 2s/step - loss: 0.3480 - accuracy: 0.8671 - val_loss: 0.3423 - val_accuracy: 0.8667
Epoch 8/50
21/21 [==

## Save results as images

In [21]:
def mkdir(path):
    if os.path.exists(path) is False:
        os.mkdir(path)

mkdir(RESULT_SAVE_PATH)
mkdir(RESULT_SAVE_PATH + '/TP')
mkdir(RESULT_SAVE_PATH + '/TN')
mkdir(RESULT_SAVE_PATH + '/FP')
mkdir(RESULT_SAVE_PATH + '/FN')

index = 0
for imgs, labels in ds_validation:
    preds = model.predict(imgs)
    for idx in range (imgs.shape[0]):
        gt = labels[idx].numpy()
        y = preds[idx]
        if gt == 1 and y > 0.5:
            path = RESULT_SAVE_PATH + '/TP'
        elif gt == 1 and y <= 0.5:
            path = RESULT_SAVE_PATH + '/FN'
        elif gt == 0 and y > 0.5:
            path = RESULT_SAVE_PATH + '/FP'
        else:
            path = RESULT_SAVE_PATH + '/TN'
        cv2.imwrite(path + '/%.f_%04d.png' % (y, index), imgs[idx].numpy() *255)
        index +=1

1/1 [==============================] - 0s 381ms/step
